# Scoring

- The maximum number of points for this assignment is 10, the minimum number of points is 0.
- You could earn 2 additional points (12 in total) if you do the additional 
exercise.
- You have two weeks to complete the assignment. Once the assignment is submitted you are not allowed to change it.
- One week delay is penalized with 1 point.
  - Example. The assignment is issued on the 1st January. The deadline without penalization is until 23:59 January 14th (anywhere on Earth). Student A submits his assignment on 22:51 January 13th and is not penalized; student B submits his assignment on 01:13 January 15th and is penalized with 1 point; student C submits his assignment on 3:56 January 23th and is penalized with 2 points and so on.




# Setting up the environment



In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Download predefined datasets
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

!wget https://github.com/iburenko/hse_course/raw/main/week3/data/corona_train_segm_dataset.npz
!wget https://github.com/iburenko/hse_course/raw/main/week3/data/corona_val_segm_dataset.npz
!wget https://github.com/iburenko/hse_course/raw/main/week3/data/radiopedia_train_segm_dataset.npz
!wget https://github.com/iburenko/hse_course/raw/main/week3/data/radiopedia_val_segm_dataset.npz

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Install required packages into the environment
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

!pip install segmentation_models_pytorch

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Import needed packages into the environment
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

from os import listdir as ls

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from natsort import natsorted

from tqdm.notebook import tqdm

import torch
import numpy as np

from segmentation_models_pytorch import Unet
from segmentation_models_pytorch.losses import DiceLoss

# Train/val loop

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Write the code for train and validation loops.
# Send dataloader for the train/validation loop to the train/val function.
# You could send any number of arguments to train and val functions in
# addition to the dataloader.
# train function should return the mean loss over the entire dataset;
# val function should return two parameters: mean dice and mean loss (over 
# the entire validation dataset).
#
# If you feel confident, you can rewrite given code and change train/val
# function (say, make them methods of some class). This is not necessary
# and is absolutely up to you. If you change the code of train/val function
# you should check that everything works correctly by yourself, 
# we won't do this!
# We will just run the entire notebook and check cells' output.
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


def train(train_loader, *args, **kwargs):
    # Your code for the trian loop here

    return mean_loss
        
def val(val_loader, *args, **kwargs):
    # Your code for the val loop here

    return mean_dice, mean_loss

# Load data into the environment

In [ ]:
corona_train = np.load('corona_train_segm_dataset.npz')
corona_val = np.load('corona_val_segm_dataset.npz')
radio_train = np.load('radiopedia_train_segm_dataset.npz')
radio_val = np.load('radiopedia_val_segm_dataset.npz')

# Define Dataset class

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Define a subclass of pytorch's Dataset class.
# In this class we have to write code (at least) 
# for three ('virtual') functions:
# __init__(self, *args, **kwargs)
# __len__(self, *args, **kwargs)
# __getitem__(self, idx, *args, **kwargs)
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


class SegmentationDatasetNumpy(Dataset):
    def __init__(self, train, dataset):
        super().__init__()
        assert train in ['train', 'val']
        assert dataset in ['corona', 'radiopedia', 'all']
        self.train = train
        if self.train == 'train':
            if dataset == 'corona':
                data = corona_train
            elif dataset == 'radiopedia':
                data = radio_train
            else:
                # additional exercise (see below)
                pass
        else:
            if dataset == 'corona':
                data = corona_val
            elif dataset == 'radiopedia':
                data = radio_val
            else:
                # additional exercise (see below)
                pass 
        self.all_data = data['images']
        self.all_masks = data['masks']

    def __len__(self):
        return self.all_data.shape[-1]
    
    def __getitem__(self, idx):
        return self.all_data[...,idx], self.all_masks[...,idx]

# Initialize Dataset and DataLoader classes

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# initialize datasets for train/val corona/radiopedia data
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

train_ds_corona = SegmentationDatasetNumpy('train', 'corona')
val_ds_corona = SegmentationDatasetNumpy('val', 'corona')
train_ds_radio = SegmentationDatasetNumpy('train', 'radiopedia')
val_ds_radio = SegmentationDatasetNumpy('val', 'radiopedia')

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# initialize DataLoader for train/val corona/radiopedia datasets
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

train_loader_corona = DataLoader(train_ds_corona, batch_size=16, shuffle=True, num_workers=8) 
val_loader_corona = DataLoader(val_ds_corona, batch_size=16, shuffle=False, num_workers=8)
train_loader_radio = DataLoader(train_ds_radio, batch_size=16, shuffle=True, num_workers=8) 
val_loader_radio = DataLoader(val_ds_radio, batch_size=16, shuffle=False, num_workers=8)

# Define Dice score, the measure of the segmnetation quality

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Define dice score for the positive class
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def calculate_dice(pred, y):
  return 2*((pred > 0)*y).sum(axis=(1,2,3)) / (((pred > 0).int() + y).sum(axis=(1,2,3)) + 1e-3)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Define the network, criterion (loss function) and the optimizer
# BEWARE! The choice of hyperparameters is very important!
# Try different values for the learning rate (1e-3, 1e-4, 1e-5, ...);
# Feel free to change the optimizer, you could try different encoder or
# use Unet++ or other versions of Unet.
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

model = Unet(encoder_name='resnet34', encoder_weights=None, in_channels=1, classes=1)
model = model.cuda()

criterion = DiceLoss('binary')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)

In [ ]:
epochs = 50 # Feel free to change the number of epochs
train_losses = np.zeros(epochs)
val_losses = np.zeros(epochs)
val_dices_domain = np.zeros(epochs)
val_dices_shift = np.zeros(epochs)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Define train/val loop
# 
# For each epoch save the mean value of the loss function for the train loop.
#
# After each train loop save the mean values of the loss function 
# and dice score for the dataset which you ***did use*** to train 
# the neural network.
#
# After each train loop save the mean values of the loss function 
# and dice score for the dataset which you ***did NOT use*** to train 
# the neural network.
# 
# Save the model with the best result obtained on a validation
# dataset (the same as was used for training).
#
# Hint: do not forget to normalize data!
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

for i in range(epochs):
    pass

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# (maximum 2 points)
# Output the best obtained dice score and 
# plot dice scores for corona and radiopedia validation datasets when trained
# on ***corona*** dataset.
# You should obtain (best) mean_dice (for corona dataset) > 0.6.
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

###
# Your code here
###

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# (maximum 2 points)
# Load the best model obtained while training 
# and run the validation loop one more time.
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

###
# Your code here
###

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# (maximum 2 points)
# Output the best obtained dice score and 
# plot dice scores for corona and radiopedia validation datasets when trained
# on ***radiopedia*** dataset
# You should obtain (best) mean_dice (for radio dataset) > 0.3.
#
# ***NotaBene***: training on radiopedia is much less stable compared to
# training on corona dataset. Nevertheless, you should be able to score 
# dice measure > 0.3.
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# (maximum 2 points)
# Load the best model obtained while training 
# and run the validation loop one more time.
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

###
# Your code here
###

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# (maximum 2 points)
# ADDITIONAL EXERCISE 
# Output the best obtained dice scores and 
# plot dice scores for corona and radiopedia validation datasets when trained
# on ***corona and radiopedia*** datasets. 
# Try to make use of a mix of two datasets and obtain better results.
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

###
# Your code here
###

### Your conclusion

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# (maximum 2 points)
# What conclusion could be made? Are we able to generalize to other datasets?
# Why?
# How do think why results on radiopedia are not so stable compared to those
# obtained on the corona dataset?
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Please add your thoughts here